# Spearman Correlation
The idea is to analyse the correlation between the disease rates and suicide rates in each municipality.

Import libraries

In [2]:
import pandas as pd
import glob
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

root = "../"

Read suicide csv

In [3]:
suicide = pd.read_csv(root + 'CSV/Suicide/suicide_rates_08_18.csv', sep=',', index_col=0)
suicide

,MUNCOD,RATE_08,RATE_09,RATE_10,RATE_11,RATE_12,RATE_13,RATE_14,RATE_15,RATE_16,RATE_17,RATE_18
0,110001,20.344224,8.212203,8.189337,4.127456,12.464166,7.773632,3.898332,11.728829,7.841292,11.793844,4.316485
1,110002,9.458389,2.338060,4.427031,4.368243,9.703818,1.974938,4.860976,4.789226,5.665936,9.315758,1.883807
2,110003,0.000000,14.936520,0.000000,0.000000,0.000000,15.396459,0.000000,31.471282,15.900779,0.000000,18.389114
3,110004,5.110972,7.626311,2.544497,1.266480,5.042229,1.164646,5.776607,6.878683,10.241588,5.649271,9.432516
4,110005,0.000000,0.000000,11.743981,0.000000,11.868028,0.000000,0.000000,11.119760,11.136478,5.576001,6.081245
...,...,...,...,...,...,...,...,...,...,...,...,...
5376,522200,7.874636,0.000000,15.937525,15.817779,15.702285,0.000000,7.494566,0.000000,0.000000,7.312614,0.000000
5377,522205,0.000000,0.000000,0.000000,0.000000,13.199578,12.605572,0.000000,12.238404,0.000000,0.000000,0.000000
5378,522220,67.249496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5379,522230,0.000000,18.660198,0.000000,0.000000,0.000000,18.315018,0.000000,0.000000,0.000000,0.000000,0.000000


In [88]:
suicide[suicide["MUNCOD"] == 355030]

,MUNCOD,RATE_08,RATE_09,RATE_10,RATE_11,RATE_12,RATE_13,RATE_14,RATE_15,RATE_16,RATE_17,RATE_18
3685,355030,4.549487,4.683992,4.722364,4.763117,4.931138,4.593181,4.488944,4.411829,3.164932,3.559948,1.741006


In [160]:
disease = ""
path = root + "CSV/TabNet/Internacoes_Rate/"
all_files = glob.glob(path + "*.csv")

years = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
suicide_df = suicide.copy()
suicide_df["SUICIDE"] = suicide_df.drop(columns="MUNCOD").sum(axis=1)/(len(suicide_df.columns) - 1)
suicide_df = suicide_df[["MUNCOD", "SUICIDE"]]
n = 71

diseases_list = []
corrs_list = []
p_value_list = []
size_list = []

for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split(".csv")[0]
    disease_df = pd.read_csv(path + disease + '.csv', sep=',', index_col=0) 
    disease_df["Total"] = disease_df.drop(columns="MUNCOD").sum(axis=1)/(len(disease_df.columns) - 1)
    disease_df = disease_df[["MUNCOD", "Total"]]
    disease_df = disease_df[(disease_df["Total"] != 0)]
    if disease_df.shape[0] > 2500:
        final_df = pd.merge(disease_df, suicide_df, on="MUNCOD")
        size_list.append(final_df.shape[0])
        corr, p_value = spearmanr(final_df["Total"],final_df["SUICIDE"])
        diseases_list.append(disease)
        corrs_list.append(corr)
        p_value_list.append(p_value)

In [165]:
corrs_data = {'Doenças': diseases_list, 'Correlação com suicidio': corrs_list, 'P value': p_value_list, "Qtd Municipios": size_list}
corrs_df = pd.DataFrame(data=corrs_data)
corrs_df.sort_values(by=['Correlação com suicidio'], ascending=False).head(50)

,Doenças,Correlação com suicidio,P value,Qtd Municipios
115,TRANSTORNOS_DE_HUMOR_[AFETIVOS],0.434420,1.360525e-154,3357
120,TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁL...,0.397686,6.620686e-131,3445
118,TRANSTORNOS_MENTAIS_E_COMPORTAMENTAIS,0.385747,2.315814e-175,4950
9,CANCER,0.374514,6.261993e-178,5355
58,NEOPLASIA_MALIGNA_DO_CÓLON,0.372220,2.342180e-105,3189
56,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,0.371460,5.702495e-89,2696
119,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,0.369459,1.201612e-90,2780
8,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,0.368975,9.287829e-156,4833
24,DPOC,0.368975,9.287829e-156,4833
63,NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL,0.352598,9.779801e-86,2903


In [181]:
# disease = ""
# path = root + "CSV/TabNet/Internacoes_Rate/"
# all_files = glob.glob(path + "*.csv")

# years = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
# n = 24
# for file in all_files[n:n+1]:
#     file_name = file.split("\\")[1]
#     disease = file_name.split(".csv")[0]
#     print(disease)
#     disease_df = pd.read_csv(path + disease + '.csv', sep=',', index_col=0)
#     year_list = []
#     corrs_list = []
#     p_value_list = []
#     size_list = []
#     for i,year in enumerate(years[:-1]):
#         col_year = "RATE_" + year
#         col_year_1 = "RATE_" + years[i+1]
#         if col_year in disease_df.columns:
#             disease_year_df = disease_df[[col_year, "MUNCOD"]]
#             disease_year_df = disease_year_df.rename(columns={col_year: disease})
#             suicide_year_df = suicide[[col_year_1, "MUNCOD"]]
#             final_df = pd.merge(suicide_year_df, disease_year_df, left_on="MUNCOD", right_on="MUNCOD")
#             final_df = final_df[final_df[disease] != 0]
#             if final_df.shape[0] > 300:
#                 corr, p_value = spearmanr(final_df[disease],final_df[col_year_1])
#                 if p_value < 0.05:
#                     year_list.append(year)
#                     corrs_list.append(corr)
#                     p_value_list.append(p_value)
#                     size_list.append(final_df.shape[0])
#     corrs_data = {'Ano': year_list, 'Correlação com suicidio': corrs_list, 'P value': p_value_list}
#     corrs_df = pd.DataFrame(data=corrs_data)
# corrs_df   
# #corrs_df["Correlação com suicidio"].mean()